In [85]:
'''
Design of a Neural Network from scratch

*************<IMP>*************
Mention hyperparameters used and describe functionality in detail in this space
- carries 1 mark
'''
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
class NN: 
    parameters = list()
    
    def init_params(self,neuron_count_per_layer):
        np.random.seed(10)
        num_layers = len(neuron_count_per_layer)
        params = [None for i in range(2*(num_layers-1))]
        for i in range(1,num_layers):
            params[2*i-2]=np.random.randn(neuron_count_per_layer[i], neuron_count_per_layer[i-1]) * 0.01
            params[2*i-1]=np.zeros((neuron_count_per_layer[i], 1))
        #print(params)
        return params
    
    #Clean the data by replacing the null values with the mean/median/mode of the column
    def data_clean(self,df):
        df.columns = df.columns.str.strip()
        for column in df.columns:
            if column in ['Weight','HB','BP']:
                df[column].fillna(value=df[column].mean(), inplace=True)
            elif column in ['Community','Delivery phase','IFA','Education']:
                df[column].fillna(value=df[column].mode()[0], inplace=True)
            else:
                df[column].fillna(value=df[column].median(), inplace=True)
        return df
    
    def relu(self,Z):
        return np.maximum(0,Z),Z
    
    def sigmoid(self,Z):
        return 1/(1+np.exp(-Z)),Z
    
    def compute_activation(self,A,weight,bias,activation):
        Z = weight@A + bias
        cache1 = (A,weight,bias)
        if activation=='relu':
            A1,cache2 = self.relu(Z)
        else:
            A1,cache2 = self.sigmoid(Z)
        return A1,(cache1,cache2)
     
    def compute_gradients(self,dA,vals,activation):
        cache1,cache2 = vals
        if activation=='relu':
            dZ = np.array(dA,copy=True)
            #print(cache2.shape,dA.shape)
            dZ[cache2<=0]=0
        if activation=='sigmoid':
            sig = 1/(1+np.exp(-cache2))
            dZ = dA * sig * (1-sig)
        A_prev, W, b = cache1
        x = A_prev.shape[1]
        dW = 1 / x * dZ @ A_prev.T
        db = 1 / x * np.sum(dZ, axis=1, keepdims=True)
        dA_prev = W.T @ dZ
        return dA_prev,dW,db
            
    
    def forward_propogation(self,X,parameters):
        num_layers = len(parameters)//2
        back_prop_values = []
        A = X
        
        for i in range(1,num_layers):
            prev_act = A
            weight = parameters[2*i-2]
            bias = parameters[2*i-1]
            A,back_prop_value = self.compute_activation(prev_act,weight,bias,activation='relu')
            back_prop_values.append(back_prop_value)
        
    
        #For Last layer i.e sigmoid 
        A,back_prop_value = self.compute_activation(A,parameters[-2],parameters[-1],activation='sigmoid')
        back_prop_values.append(back_prop_value) 
        #print(len(back_prop_values))
        return A,back_prop_values
    
    def back_propogation(self,Y,activations,parameters,back_prop_values,alpha):
        #Computing the necessary derivatives
        gradients = {}
        num_layers = len(parameters)//2
        Y.reshape(activations.shape)
        dA = - (np.divide(Y, activations) - np.divide(1 - Y, 1 - activations))
        #print(dA.shape)
        vals = back_prop_values[num_layers-1]
        gradients["dA" + str(num_layers-1)], gradients["dW" + str(num_layers)], gradients["db" + str(num_layers)] = self.compute_gradients(dA,vals,'sigmoid')
        for layer in reversed(range(num_layers-1)):
            vals = back_prop_values[layer]
            gradients["dA" + str(layer)], gradients["dW" + str(layer + 1)], gradients["db" + str(layer + 1)] = self.compute_gradients(gradients['dA'+str(layer+1)], vals, 'relu')
        
        #Updating the parameters
        for i in range(1,num_layers+1):
            parameters[2*i-2]=parameters[2*i-2] - alpha * gradients['dW'+str(i)]
            parameters[2*i-1]=parameters[2*i-1] - alpha * gradients['db'+str(i)]
        
        return parameters
        

    def calc_cost(self,A,Y):
        return np.squeeze(-1 / len(Y) * np.sum(Y * np.log(A) + (1-Y) * np.log(1-A)))

    ''' X and Y are dataframes '''

    def fit(self,X,Y):
        '''
        Function that trains the neural network by taking x_train and y_train samples as input
        '''

        #Clean the data
        X = self.data_clean(X)
        #Set hyperparameters
        num_itertations = 20000
        alpha = 0.1
        
    
        #Init parameters
        neuron_count_per_layer = [9,30,30,25,1]
        self.parameters = self.init_params(neuron_count_per_layer)
        
        #Making necessary changes to dimensions
        X = np.transpose(np.array(X))
        Y = np.array(Y)
        Y = np.reshape(Y,(1,Y.shape[0]))
        
        for i in range(1,num_itertations+1):
            #Fp
            activations,back_prop_values = self.forward_propogation(X,self.parameters)
            #Bp
            self.parameters = self.back_propogation(Y,activations,self.parameters,back_prop_values,alpha)
            #Print Cost after every 500 iters
            if i%500==0:
                print('Cost after iter '+str(i)+ ':' + str(self.calc_cost(activations,Y)/100))
 
        
        
        
    
    def predict(self,X):

        """
        The predict function performs a simple feed forward of weights
        and outputs yhat values 

        yhat is a list of the predicted value for df X
        """
        yhat = []
        X = self.data_clean(X)
        X = np.transpose(np.array(X))
        yhat = self.forward_propogation(X,self.parameters)[0][0]
        return yhat

    def CM(self,y_test,y_test_obs):
        '''
        Prints confusion matrix 
        y_test is list of y values in the test dataset
        y_test_obs is list of y values predicted by the model

        '''

        for i in range(len(y_test_obs)):
            if(y_test_obs[i]>0.6):
                y_test_obs[i]=1
            else:
                y_test_obs[i]=0

        cm=[[0,0],[0,0]]
        fp=0
        fn=0
        tp=0
        tn=0

        for i in range(len(y_test)):
            if(y_test[i]==1 and y_test_obs[i]==1):
                tp=tp+1
            if(y_test[i]==0 and y_test_obs[i]==0):
                tn=tn+1
            if(y_test[i]==1 and y_test_obs[i]==0):
                fp=fp+1
            if(y_test[i]==0 and y_test_obs[i]==1):
                fn=fn+1
        cm[0][0]=tn
        cm[0][1]=fp
        cm[1][0]=fn
        cm[1][1]=tp

        p= tp/(tp+fp)
        r=tp/(tp+fn)
        f1=(2*p*r)/(p+r)
        acc = (tp+tn)/(tp+tn+fp+fn)
        print("Confusion Matrix : ")
        print(cm)
        print("\n")
        print(f"Precision : {p}")
        print(f"Recall : {r}")
        print(f"F1 SCORE : {f1}")
        print(f"Accuracy : {acc}")
   






In [45]:
import pandas as pd
df = pd.read_csv('LBW_Dataset.csv')
X = df.iloc[:, :-1]
y = df.iloc[:, -1]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 0)


In [86]:
nn = NN()

In [87]:
nn.fit(X_train,y_train)

Cost after iter 500:0.3358885771696138
Cost after iter 1000:0.32902573716088773
Cost after iter 1500:0.3223572487603984
Cost after iter 2000:0.3247240251737678
Cost after iter 2500:0.32054758322336996
Cost after iter 3000:0.33372565704142515
Cost after iter 3500:0.317712172811239
Cost after iter 4000:0.32802505401688103
Cost after iter 4500:0.3111549532066966
Cost after iter 5000:0.3083614443556818
Cost after iter 5500:0.29647190740753404
Cost after iter 6000:0.28138442528854274
Cost after iter 6500:0.3011161345796595
Cost after iter 7000:0.29902958955491427
Cost after iter 7500:0.3398434189931012
Cost after iter 8000:0.3045896461096042
Cost after iter 8500:0.25768540932129946
Cost after iter 9000:0.30010387257492244


C:\Users\Sairam\Anaconda3\lib\site-packages\ipykernel_launcher.py:92: RuntimeWarning: invalid value encountered in true_divide
C:\Users\Sairam\Anaconda3\lib\site-packages\ipykernel_launcher.py:56: RuntimeWarning: invalid value encountered in less_equal


Cost after iter 9500:nan
Cost after iter 10000:nan
Cost after iter 10500:nan
Cost after iter 11000:nan
Cost after iter 11500:nan
Cost after iter 12000:nan
Cost after iter 12500:nan
Cost after iter 13000:nan
Cost after iter 13500:nan
Cost after iter 14000:nan
Cost after iter 14500:nan
Cost after iter 15000:nan
Cost after iter 15500:nan
Cost after iter 16000:nan


KeyboardInterrupt: 

In [83]:
y_hat = nn.predict(X_test)
print(y_hat)

C:\Users\Sairam\Anaconda3\lib\site-packages\pandas\core\generic.py:6130: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)


[0.6260476  0.61793145 0.81078178 0.98479414 0.53962577 0.4603836
 0.70512989 0.71226076 0.68330888 0.95158199 0.71837652 0.74416737
 0.93153211 0.00796239 0.34380006 0.46015509 0.82315048 0.87611452
 0.60209598 0.43137929 0.99916847 0.97358587 0.83416654 0.54130124
 0.66203327 0.52516099 0.16926046 0.91310644 0.80481914]


In [84]:
nn.CM(y_test.tolist(),y_hat)

Confusion Matrix : 
[[4, 5], [4, 16]]


Precision : 0.7619047619047619
Recall : 0.8
F1 SCORE : 0.7804878048780488
Accuracy : 0.6896551724137931


In [78]:
y_hat

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 0., 0., 1.,
       1., 0., 0., 1., 1., 1., 1., 1., 1., 0., 1., 1.])